In [1]:
import torch 
from dataset import Pic_to_Pic_dataset
from models import UNET, U2NET
from torch.utils.data import DataLoader
from loss import SSIM_DICE_BCE, DiceScore
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from PIL import Image 
from torchvision.transforms import ToTensor, ToPILImage
import os 
from tqdm import tqdm 
import cv2
from typing import List

torch.set_grad_enabled(False)

In [2]:
  

def path_gen_fn(paths: List[str], module_names: List[str] = ['ch', 'down1', 'down2', 'down3', 'down4', 'up1', 'up2', 'up3', 'up4', 'out']): 
    for path in paths: 
        for module_name in module_names: 
            yield path + module_name + '/'

In [3]:
batch_num = 0
def forward_hook(module, input, output): 
    for depth, op in enumerate(output): 
        # print(f'{op.shape = }')
        os.makedirs('plots/{}/{}'.format(patient_id, batch_num+depth), exist_ok=True)
        for idx, img in enumerate(op): 
            path = os.path.join('plots', patient_id, batch_num+depth)
            path = os.path.join(path, str(idx), module.name) 
            idx = len(os.listdir(path))

            os.makedirs(path, exist_ok=True)
            img = (img-img.min())/(img.max()-img.min())
            # print(f'{path = }')
            ToPILImage()(img).save(path+ str(idx) + '.png')
    batch_num += depth 

In [4]:
model = UNET().to('cuda:1')
model.assign_names()
ckpt = torch.load('./ckpts/quantum_noise/56/best_unet.pth') 
model.load_state_dict(ckpt['model_state'])
dice_score = DiceScore()
# print(model)


model.ch.register_forward_hook(forward_hook)
model.down1.register_forward_hook(forward_hook)
model.down2.register_forward_hook(forward_hook)
model.down3.register_forward_hook(forward_hook)
model.down4.register_forward_hook(forward_hook)
model.up1.register_forward_hook(forward_hook)
model.up2.register_forward_hook(forward_hook)
model.up3.register_forward_hook(forward_hook)
model.up4.register_forward_hook(forward_hook)
model.out.register_forward_hook(forward_hook)

Assigning names to layers


In [5]:
def imgs_to_vid(path): 
    imgs = os.listdir(path)
    imgs = [os.path.join(path, img) for img in imgs if img.endswith('.png')]
    imgs = sorted(imgs, key=lambda x: int(x.split('/')[-1].split('.')[0]))
    assert len(imgs) > 0, 'No images found in {}'.format(path)
    img = cv2.imread(imgs[0])
    video = cv2.VideoWriter('{}/vid.mp4'.format(path), cv2.VideoWriter_fourcc(*'mp4v'), 10, (img.shape[1], img.shape[0]))
    
    for img_path in tqdm(imgs): 
        if not img_path.endswith('.png'): 
            continue
        img = cv2.imread(img_path)
        video.write(img)       
    video.release()
    os.system('rm path + *.png')



In [6]:
whole_df = pd.read_csv('/home/shivac/qml-data/csv_files/val_10_org.csv') 
patient_ids = whole_df['patient_id'].unique()
patient_id = np.random.choice(patient_ids)
img_paths = whole_df[whole_df['patient_id'] == patient_id]['img_path'].values
img_paths = [os.path.join('/home/shivac/qml-data/', img_path) for img_path in img_paths]
batch_img = list() 
for img_path in img_paths: 
    img = Image.open(img_path).convert('L') 
    img = ToTensor()(img)
    batch_img.append(img)
batch_img = torch.stack(batch_img)
batch_img = batch_img.split(12)
path_gen = path_gen_fn(['plots/{}/{}/'.format(patient_id, i) for i in range(len(img_path))])

print(f'{len(batch_img)}')
with torch.no_grad():
    for img in tqdm(batch_img): 
        logits = model(img.to('cuda:1'))

26


  0%|          | 0/26 [00:00<?, ?it/s]/home/shivac/anaconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
  0%|          | 0/26 [00:37<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
whole_df = pd.read_csv('/home/shivac/qml-data/csv_files/val_10_org.csv') 
patient_ids = np.unique(whole_df.patient_id)
model.eval()
for patient_id in patient_ids:
    os.makedirs('plots/{}'.format(patient_id), exist_ok=True)
    df = whole_df[whole_df.patient_id == patient_id].sort_values('idx')
    df.reset_index(inplace=True)
    path_gen = path_gen_fn(['plots/{}/{}/'.format(patient_id, i) for i in range(len(df))])
    for i in tqdm(range(len(df))): 
        os.makedirs('plots/{}/{}'.format(patient_id, i), exist_ok=True)
        img_path = '/home/shivac/qml-data/' + df.loc[i].img_path
        mask_path = '/home/shivac/qml-data/' + df.loc[i].mask_path
        img = Image.open(img_path).convert('L')
        mask = Image.open(mask_path) 
        mask = ToTensor()(mask).unsqueeze(0)
        img = ToTensor()(img).unsqueeze(0)
        logits = model(img.cuda())
        dice = round(dice_score(mask.cuda(), logits).item(), 2)

        plt.figure(figsize=(10, 6), facecolor='gray')
        plt.axis('off')
        plt.title('Depth: ' + str(i) + ' dice_score: ' + str(dice))
        plt.subplot(1,3,1)
        plt.title('img')
        plt.axis('off')
        plt.imshow(img[0].permute(1,2,0), cmap='gray')
        plt.subplot(1,3,2)
        plt.title('mask')
        plt.axis('off')
        plt.imshow(mask[0].permute(1,2,0), cmap='gray')
        plt.subplot(1,3,3)
        plt.title('logits')
        plt.axis('off')
        plt.imshow(logits[0].detach().cpu().permute(1,2,0), cmap='gray')
        plt.savefig('plots/{}/{}/res.png'.format(patient_id, i))
        plt.clf() 
        plt.close()
        for path in os.listidir('plots/{}/{}'.format(patient_id, i)):
            imgs_to_vid(os.path.join(path, 'ch/'))
    break
        
        


  0%|          | 0/301 [00:00<?, ?it/s]

/home/shivac/anaconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
 31%|███       | 92/301 [04:47<10:53,  3.13s/it]


KeyboardInterrupt: 

In [ ]:
patient_id

'MEDVID0076_M_20211124_103754_0001_IMAGES'